In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "drive/MyDrive"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os
from PIL import Image
from matplotlib.image import imread
from os import listdir
from sklearn.preprocessing import label_binarize, LabelBinarizer
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, array_to_img
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Activation
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras.utils import to_categorical

In [ ]:
def convert_img_to_array(img_dir):
  try:
    image = cv2.imread(img_dir)
    if image is not None:
      image = cv2.resize(image, (256, 256))
      return image
    else:
      return np.array([])
  except Exception as e:
    print(e)
    return None

In [ ]:
dir = "drive/MyDrive/CNN_Plant_Data"
root_dir = listdir(dir)
image_list, label_list = [],[]
all_labels = ['Corn-Common_rust','Potato-Early_blight','Tomato-Bacterial_spot']
binary_labels = [0,1,2]
temp = -1



for dir1 in root_dir:
  plant_image_list = listdir(f"{dir}/{dir1}")
  temp = temp+1
  for image in plant_image_list:
    image_path = f"{dir}/{dir1}/{image}"
    image_list.append(convert_img_to_array(image_path))
    label_list.append(binary_labels[temp])

In [ ]:
label_counts = pd.DataFrame(label_list).value_counts()
label_counts

0    300
1    300
2    300
Name: count, dtype: int64

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.2, random_state=10)

In [ ]:
x_train = np.array(x_train, dtype=np.float16) / 225.0
x_test = np.array(x_test, dtype=np.float16) / 225.0
x_train = x_train.reshape( -1, 256,256,3)
x_test = x_test.reshape( -1, 256,256,3)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding="same",input_shape=(256,256,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),padding="same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(3,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 262144)            0         
                                                                 
 dense (Dense)               (None, 64)                1

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.0001),metrics=['accuracy'])

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

In [ ]:
epochs = 100
batch_size = 128
history = model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs,validation_data = (x_val, y_val))

Epoch 1/100
5/5 [==============================] - 16s 1s/step - loss: 0.9292 - accuracy: 0.5747 - val_loss: 0.4647 - val_accuracy: 0.7569
Epoch 2/100
5/5 [==============================] - 1s 255ms/step - loss: 0.3937 - accuracy: 0.8438 - val_loss: 0.2876 - val_accuracy: 0.9444
Epoch 3/100
5/5 [==============================] - 2s 350ms/step - loss: 0.2430 - accuracy: 0.9375 - val_loss: 0.2087 - val_accuracy: 0.9444
Epoch 4/100
5/5 [==============================] - 2s 363ms/step - loss: 0.1651 - accuracy: 0.9618 - val_loss: 0.2297 - val_accuracy: 0.9167
Epoch 5/100
5/5 [==============================] - 2s 348ms/step - loss: 0.1463 - accuracy: 0.9479 - val_loss: 0.1372 - val_accuracy: 0.9583
Epoch 6/100
5/5 [==============================] - 2s 325ms/step - loss: 0.0998 - accuracy: 0.9722 - val_loss: 0.1239 - val_accuracy: 0.9653
Epoch 7/100
5/5 [==============================] - 1s 297ms/step - loss: 0.0728 - accuracy: 0.9861 - val_loss: 0.1060 - val_accuracy: 0.9653
Epoch 8/100
5/5

In [ ]:
model.save("plant.h5")